# Wines Points prediction 

Submission Date : 3.6.2023
Task: Predict the wine score given the inputs
Instructions:
 * Use logistic regression as benchmark model
 * Use sklearn pipeliens + cv + grid search with sklearn models (e.g. KNNs, RandomForest, etc.)
 * Compare all models on proper metric (your choice)

For DNN course project:
* Use sklearn pipeliens with tensorflow models (w/wo embeddings, LSTMs, RNNs, Transformers etc.)
* Compare all models on proper metric (your choice)

In [1]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.append('../')

Here we will try to predict the points a wine will get based on known characteristics (i.e. features, in the ML terminology). The mine point in this stage is to establish a simple, ideally super cost effective, basline.
In the real world there is a tradeoff between complexity and perforamnce, and the DS job, among others, is to present a tradeoff tables of what performance is achivalbel at what complexity level. 

to which models with increased complexity and resource demands will be compared. Complexity should then be translated into cost. For example:
 * Compute cost 
 * Maintenance cost
 * Serving costs (i.e. is new platform needed?) 
 

## Loading the data

In [2]:
import pandas as pd
import cufflinks as cf; cf.go_offline()

In [3]:
wine_reviews = pd.read_csv("../../data/winemag-data-130k-v2.csv")
wine_reviews.shape

(129971, 14)

In [4]:
wine_reviews.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [5]:
wine_reviews.region_1.value_counts()

Napa Valley                4480
Columbia Valley (WA)       4124
Russian River Valley       3091
California                 2629
Paso Robles                2350
                           ... 
Lamezia                       1
Trentino Superiore            1
Grave del Friuli              1
Vin Santo di Carmignano       1
Paestum                       1
Name: region_1, Length: 1229, dtype: int64

In [6]:
# iv'e noticed that california which is supposed to be province was also region
# investigated the data to see that there is no corruption 
wine_reviews.loc[wine_reviews['region_1'] == 'California']

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
68,68,US,"Very deep in color and spicy-smoky in flavor, ...",NaN,86,12.0,California,California,California Other,Jim Gordon,@gordone_cellars,Cocobon 2014 Red (California),Red Blend,Cocobon
396,396,US,"Rich in spicy, briary berry-cherry and chocola...",Ancient Vines,85,16.0,California,California,California Other,NaN,NaN,Cline 2007 Ancient Vines Zinfandel (California),Zinfandel,Cline
403,403,US,"Dry and acidic, this light-bodied wine has tar...",Le Bon Viveur,85,15.0,California,California,California Other,NaN,NaN,Vinavanti 2005 Le Bon Viveur Red (California),Red Blend,Vinavanti
411,411,US,A nice Cabernet for this everyday price. Dry a...,Split Oak Vineyard,85,12.0,California,California,California Other,NaN,NaN,Morro Bay 2006 Split Oak Vineyard Cabernet Sau...,Cabernet Sauvignon,Morro Bay
424,424,US,"This wine is dry and rather full bodied, start...",Verdigris,89,30.0,California,California,California Other,Jim Gordon,@gordone_cellars,Las Positas 2014 Verdigris White (California),White Blend,Las Positas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129616,129616,US,"Very sweet, but the raspberry and cherry fruit...",Dessert Wine,84,18.0,California,California,California Other,NaN,NaN,Martin Ray 2008 Dessert Wine Red (California),Red Blend,Martin Ray
129689,129689,US,"Vanilla cream, honey and tropical-fruit flavor...",New Generation Unoaked,87,23.0,California,California,California Other,NaN,NaN,Passaggio 2011 New Generation Unoaked Chardonn...,Chardonnay,Passaggio
129694,129694,US,This is one of Chandon's sweeter bubblies. It ...,Brut Classic,87,22.0,California,California,California Other,NaN,NaN,Chandon NV Brut Classic Sparkling (California),Sparkling Blend,Chandon
129695,129695,US,"Honey-sweet flavors of raspberries, pink grape...",Rosé,87,22.0,California,California,California Other,NaN,NaN,Chandon NV Rosé Sparkling (California),Sparkling Blend,Chandon


# Data Preparation

In [7]:
wine_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


In [8]:
# index column, not necessary
wine_reviews = wine_reviews.drop(columns='Unnamed: 0')

In [9]:
# removed all of the full duplicated rows, kept only one appearance of the row
wine_reviews = wine_reviews.drop_duplicates(keep='first')

In [10]:
# null prices replaced by price average of the country
wine_reviews['price'].fillna(wine_reviews.groupby(['country'])['price'].transform('mean'), inplace = True)

In [11]:
wine_reviews.isnull().sum()


country                     59
description                  0
designation              34545
points                       0
price                        5
province                    59
region_1                 19560
region_2                 73219
taster_name              24917
taster_twitter_handle    29446
title                        0
variety                      1
winery                       0
dtype: int64

In [12]:
wine_reviews.country.value_counts()

US                        50457
France                    20353
Italy                     17940
Spain                      6116
Portugal                   5256
Chile                      4184
Argentina                  3544
Austria                    3034
Australia                  2197
Germany                    1992
South Africa               1301
New Zealand                1278
Israel                      466
Greece                      432
Canada                      226
Bulgaria                    132
Hungary                     129
Romania                     102
Uruguay                      98
Turkey                       81
Slovenia                     77
Georgia                      76
Croatia                      70
Mexico                       68
England                      63
Moldova                      56
Brazil                       49
Lebanon                      32
Morocco                      24
Peru                         16
Ukraine                      14
Czech Re

In [13]:
# wine_reviews.price.plot()
# removing price outliers
wine_reviews = wine_reviews[(wine_reviews ["price"] < wine_reviews["price"].quantile(0.99)) & (wine_reviews ["price"] > wine_reviews["price"].quantile(0.01))]

In [14]:
value_counts = wine_reviews['country'].value_counts()

mask = wine_reviews['country'].isin(value_counts[value_counts > 50].index)

wine_reviews= wine_reviews[mask]

wine_reviews.count()

country                  116870
description              116870
designation               83325
points                   116870
price                    116870
province                 116870
region_1                  98268
region_2                  45989
taster_name               92523
taster_twitter_handle     88079
title                    116870
variety                  116869
winery                   116870
dtype: int64

In [15]:
(wine_reviews['variety'].value_counts() > 50).sum()

120

In [16]:
value_counts = wine_reviews['variety'].value_counts()

mask = wine_reviews['variety'].isin(value_counts[value_counts > 50].index)

wine_reviews= wine_reviews[mask]

wine_reviews.count()

country                  111931
description              111931
designation               79700
points                   111931
price                    111931
province                 111931
region_1                  94799
region_2                  45165
taster_name               88435
taster_twitter_handle     84284
title                    111931
variety                  111931
winery                   111931
dtype: int64

In [17]:
wine_reviews['year'] = wine_reviews.title.str.extract(r'(\b\d{4}\b)')
wine_reviews['year'] = wine_reviews['year'].astype('Int64')
wine_reviews['year'] = wine_reviews['year'].where(wine_reviews['year'].between(1970, 2023))

In [18]:
wine_reviews[wine_reviews.year.isnull()]

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year
63,France,"This fat, yeasty Champagne is comprised predom...",Brut Rosé,86,58.0,Champagne,Champagne,NaN,Roger Voss,@vossroger,Roland Champion NV Brut Rosé (Champagne),Champagne Blend,Roland Champion,<NA>
69,France,"This cooperative, based in Aÿ, has benefited f...",Brut Rosé,86,55.0,Champagne,Champagne,NaN,Roger Voss,@vossroger,Collet NV Brut Rosé (Champagne),Champagne Blend,Collet,<NA>
237,Italy,The wine boasts a beautiful appearance thanks ...,NaN,85,18.0,Central Italy,San Marino,NaN,NaN,NaN,Consorzio Vini Tipici di San Marino NV Moscato...,Moscato,Consorzio Vini Tipici di San Marino,<NA>
315,Italy,"Honey, almond blossom and citrus zest are foll...",Extra Dry,86,15.0,Veneto,Prosecco di Valdobbiadene,NaN,NaN,NaN,Bellussi NV Extra Dry (Prosecco di Valdobbiad...,Prosecco,Bellussi,<NA>
326,Italy,Drying mineral tones face off with luscious pe...,Extra Dry,86,15.0,Veneto,Prosecco di Valdobbiadene,NaN,NaN,NaN,Marsuret NV Extra Dry (Prosecco di Valdobbiad...,Prosecco,Marsuret,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129719,France,In order to maintain freshness and crispness i...,Domaine Vieilles Vignes Brut,90,49.0,Champagne,Champagne,NaN,Roger Voss,@vossroger,Bruno Gobillard NV Domaine Vieilles Vignes Bru...,Champagne Blend,Bruno Gobillard,<NA>
129743,France,"Crisp and fruity, this mature-tasting wine has...",Réserve Brut,90,70.0,Champagne,Champagne,NaN,Roger Voss,@vossroger,Castelnau NV Réserve Brut (Champagne),Champagne Blend,Castelnau,<NA>
129791,US,"Floral and fruity, with hints of peach, muskme...",NaN,89,18.0,New York,Finger Lakes,Finger Lakes,Anna Lee C. Iijima,NaN,Hazlitt 1852 Vineyards 2011 Sauvignon Blanc (F...,Sauvignon Blanc,Hazlitt 1852 Vineyards,<NA>
129869,US,"This wine hits the mark, combining intriguing ...",Brut,90,25.0,California,California,California Other,Jim Gordon,@gordone_cellars,Biltmore Estate NV Brut Sparkling (California),Sparkling Blend,Biltmore Estate,<NA>


In [19]:
print(wine_reviews.variety.unique())
white_wines = ['White Blend', 'Pinot Gris', 'Riesling', 'Gewürztraminer', 'Chardonnay', 'Chenin Blanc', 'Sauvignon Blanc',
               'Grillo', 'Albariño', 'Verdejo', 'Fumé Blanc', 'Vermentino', 'Grenache Blanc', 'Pinot Blanc', 'Viognier',
               'Grenache Blanc', 'Grüner Veltliner', 'Verdicchio', 'Sémillon', 'Fiano', 'Ribolla Gialla', 'Verdelho',
               'Marsanne', 'Glera', 'Viura', 'Greco', 'Verdicchio', 'Falanghina', 'Arinto', 'Arneis', 'Godello', 'Vidal Blanc']

red_wines = ['Portuguese Red', 'Pinot Noir', 'Cabernet Sauvignon', 'Nerello Mascalese', 'Malbec', 'Tempranillo Blend',
             'Red Blend', 'Merlot', "Nero d'Avola", 'Sangiovese', 'Cabernet Franc', 'Bordeaux-style Red Blend', 'Aglianico',
             'Petite Sirah', 'Touriga Nacional', 'Carmenère', 'Zinfandel', 'Grüner Veltliner', 'Grenache', 'Blaufränkisch',
             'Sagrantino', 'Shiraz', 'Montepulciano', 'Barbera', 'Rhône-style Red Blend', 'Tempranillo', 'Mencía',
             'Zweigelt', 'Dolcetto', 'Pinotage', 'Tannat', 'Garnacha', 'Negroamaro', 'Mourvèdre', 'Austrian Red Blend',
             'Moscatel', 'Malbec-Merlot', 'Cabernet Blend', 'Spätburgunder', 'Tinto Fino', 'Malbec-Cabernet Sauvignon',
             'Carignan']

rose_wines = ['Rosé', 'Rosato', 'Provence red blend', 'Rosado']

sparkling_wines = ['Champagne Blend', 'Prosecco', 'Sparkling Blend', 'Portuguese Sparkling', 'Sparkling Wine']

# Create a new column wine_category based on the grape type
wine_reviews['wine_category'] = ''
wine_reviews.loc[wine_reviews['variety'].isin(white_wines), 'wine_category'] = 'White'
wine_reviews.loc[wine_reviews['variety'].isin(red_wines), 'wine_category'] = 'Red'
wine_reviews.loc[wine_reviews['variety'].isin(rose_wines), 'wine_category'] = 'Rosé'
wine_reviews.loc[wine_reviews['variety'].isin(sparkling_wines), 'wine_category'] = 'Sparkling'

wine_reviews.head()

['White Blend' 'Portuguese Red' 'Pinot Gris' 'Riesling' 'Pinot Noir'
 'Gewürztraminer' 'Cabernet Sauvignon' 'Nerello Mascalese' 'Chardonnay'
 'Malbec' 'Tempranillo Blend' 'Meritage' 'Red Blend' 'Merlot'
 "Nero d'Avola" 'Chenin Blanc' 'Gamay' 'Sauvignon Blanc' 'Primitivo'
 'Petit Verdot' 'Bordeaux-style White Blend' 'Grillo' 'Sangiovese'
 'Cabernet Franc' 'Champagne Blend' 'Bordeaux-style Red Blend' 'Aglianico'
 'Petite Sirah' 'Touriga Nacional' 'Carmenère' 'Albariño' 'Petit Manseng'
 'Rosé' 'Zinfandel' 'Vernaccia' 'Rosato' 'Grüner Veltliner' 'Viognier'
 'Vermentino' 'Grenache Blanc' 'Syrah' 'Nebbiolo' 'Pinot Blanc' 'Barbera'
 'Rhône-style Red Blend' 'Portuguese White' 'Sauvignon'
 'Sangiovese Grosso' 'Torrontés' 'G-S-M' 'Verdejo' 'Fumé Blanc'
 'Pinot Bianco' 'Bonarda' 'Shiraz' 'Montepulciano' 'Moscato' 'Grenache'
 'Blaufränkisch' 'Friulano' 'Sagrantino' 'Cabernet Sauvignon-Syrah'
 'Prosecco' 'Sparkling Blend' 'Muscat' 'Garganega' 'Pinot Grigio'
 'Tempranillo' 'Cortese' 'Mencía' 'Zweige

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year,wine_category
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,39.928286,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,White
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.000000,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,Red
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,White
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,White
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,Red


In [20]:
# make sure that there is not testers without name but with twitter handler
wine_reviews[wine_reviews.taster_twitter_handle.notna() & wine_reviews.taster_name.isna()]

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year,wine_category


In [21]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [22]:
wine_reviews['description'] = wine_reviews['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
wine_reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year,wine_category
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,39.928286,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,White
1,Portugal,"This ripe fruity, wine smooth still structured...",Avidagos,87,15.000000,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,Red
2,US,"Tart snappy, flavors lime flesh rind dominate....",NaN,87,14.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,White
3,US,"Pineapple rind, lemon pith orange blossom star...",Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,White
4,US,"Much like regular bottling 2012, comes across ...",Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,Red


In [23]:
wine_reviews.dropna(subset=["variety"])

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year,wine_category
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,39.928286,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,White
1,Portugal,"This ripe fruity, wine smooth still structured...",Avidagos,87,15.000000,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,Red
2,US,"Tart snappy, flavors lime flesh rind dominate....",NaN,87,14.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,White
3,US,"Pineapple rind, lemon pith orange blossom star...",Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,White
4,US,"Much like regular bottling 2012, comes across ...",Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,Red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,Germany,Notes honeysuckle cantaloupe sweeten delicious...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.000000,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef),2013,White
129967,US,Citation given much decade bottle age prior re...,NaN,90,75.000000,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation,2004,Red
129968,France,Well-drained gravel soil gives wine crisp dry ...,Kritt,90,30.000000,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,2013,White
129969,France,"A dry style Pinot Gris, crisp acidity. It also...",NaN,90,32.000000,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss,2012,White


In [24]:
wine_reviews["region_1"] = wine_reviews["region_1"].fillna('Unknown')

In [25]:
wine_reviews["region_2"] = wine_reviews["region_2"].fillna('Unknown')

In [26]:
wine_reviews["taster_twitter_handle"] = wine_reviews["taster_twitter_handle"].fillna('Unknown')

In [27]:
wine_reviews["designation"] = wine_reviews["designation"].fillna('Unknown')

In [28]:
wine_reviews["taster_name"] = wine_reviews["taster_name"].fillna('Unknown')

In [29]:
# remove special characters
def remove_special_chars(text):
    special_characters=['@', '#', '$', '*', '&', '?', '!', ',', '.']
    for i in special_characters:
        text = text.replace(i,"")
    return text

wine_reviews["description"] = wine_reviews["description"].apply(remove_special_chars)
wine_reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year,wine_category
0,Italy,Aromas include tropical fruit broom brimstone ...,Vulkà Bianco,87,39.928286,Sicily & Sardinia,Etna,Unknown,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,White
1,Portugal,This ripe fruity wine smooth still structured ...,Avidagos,87,15.000000,Douro,Unknown,Unknown,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,Red
2,US,Tart snappy flavors lime flesh rind dominate S...,Unknown,87,14.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,White
3,US,Pineapple rind lemon pith orange blossom start...,Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,Unknown,Alexander Peartree,Unknown,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,White
4,US,Much like regular bottling 2012 comes across r...,Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,Red


In [30]:
wine_reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year,wine_category
0,Italy,Aromas include tropical fruit broom brimstone ...,Vulkà Bianco,87,39.928286,Sicily & Sardinia,Etna,Unknown,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,White
1,Portugal,This ripe fruity wine smooth still structured ...,Avidagos,87,15.000000,Douro,Unknown,Unknown,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,Red
2,US,Tart snappy flavors lime flesh rind dominate S...,Unknown,87,14.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,White
3,US,Pineapple rind lemon pith orange blossom start...,Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,Unknown,Alexander Peartree,Unknown,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,White
4,US,Much like regular bottling 2012 comes across r...,Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,Red


In [31]:
wine_reviews.to_csv('clean_wine_reviews_data.csv')

# Pipelines

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

y = pd.DataFrame(wine_reviews, columns=['points'])
x = wine_reviews.drop('points', axis=1)
categorical_features = ['country', 'taster_name','variety','designation','province','winery']
numerical_features = ['price']
text_feature = 'description'

category_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehotencoding', OneHotEncoder(handle_unknown='ignore'))
])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('standardscaler', StandardScaler())
])

text_tansformer = Pipeline(steps=[
    ('countvectorizer', CountVectorizer(max_features = 1500))
    
])

preprocessing = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numerical_features),
        ('category', category_transformer, categorical_features),
        ('text',text_tansformer , text_feature) 
    ], remainder='drop')

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessing),
    ('regressor', LinearRegression())
])

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=29)

# pipeline.fit(X_train, y_train)
# prediction = pipeline.predict(X_test)
# mse = mean_squared_error(y_test, prediction)

# print(f'linear regression mse score is {mse}')


print(X_train.shape ,y_train.shape, X_test.shape,y_test.shape )

param_grid = [{
    'regressor': [LinearRegression()],
    'regressor__fit_intercept': [True, False]
},{
    'regressor': [DecisionTreeRegressor()],
    'regressor__max_depth': [5, 10, 15],
    'regressor__min_samples_leaf':[10, 20, 30], 
},{
    'clasiffier': [KNeighborsRegressor()],
    'classifier__n_neighbors': [10, 20, 40, 100],
    'classifier__weights': ['uniform', 'distance']
}]

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Get the best model and its MSE
best_model = grid_search.best_estimator_
best_mse = -grid_search.best_score_

print("Best Model:")
print(best_model)
print("Best MSE:", best_mse)

# Evaluate the model on the test set
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)

In [ ]:
1-2 layers up to 30 neurons